# Interactive Demo of the Model on Functions Integration

In [1]:
import os

import numpy
import pandas
import sympy
import torch

from src.envs import build_env
from src.model import build_modules
from src.utils import AttrDict
from src.utils import to_cuda

## Build Environment - Reload Model

Get Trained Model:

In [2]:
model_path = '../models/integrations/fwd_bwd_model.pth'
assert os.path.isfile(model_path)

Set the Parameters for Environment and for the Model:

 Environment:
- **env_name**: SymPy character environment.
- **int_base**: integer representation base.
- **balanced**: balanced representation (base > 0).
- **positive**: do not sample negative numbers.
- **precision**: float numbers precision.
- **n_variables**: number of variables in expressions (between 1 and 4).
- **n_coefficients**: number of coefficients in expressions (between 0 and 10).
- **leaf_probs**: leaf probabilities of being a variable, a coefficient, an integer, or a constant.
- **max_len**: maximum sequences length.
- **max_int**: maximum integer value.
- **max_ops**: maximum number of operators.
- **max_ops_G**: maximum number of operators for G in IBP.
- **clean_prefix_expr**: clean prefix expressions (f x -> Y, derivative f x x -> Y').
- **rewrite_functions**: rewrite expressions with SymPy.
- **tasks**: tasks (prim_fwd, prim_bwd, prim_ibp, ode1, ode2).
- **operators**: operators (add, sub, mul, div), followed by weight.


 Model:
- **cpu**: run on CPU.
- **emb_dim**: embedding layer size.
- **n_enc_layers**: number of transformer layers in the encoder.
- **n_dec_layers**: number of transformer layers in the decoder.
- **n_heads**: number of transformer heads.
- **dropout**: dropout.
- **attention_dropout**: dropout in the attention layer.
- **sinusoidal_embeddings**: use sinusoidal embeddings.
- **share_inout_emb**: share input and output embeddings.
- **reload_model**: reload a pretrained model.

In [3]:
params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_fwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

Set the Environment with SymPy:

In [4]:
env = build_env(params)
x = env.local_dict['x']

Build Model Modules:

In [5]:
modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

## Use the Model for finding the Integral of a Function

Start from a function $F$, compute its derivative $f = F'$, and try to recover $F$ from $f$.

Declare $F$, the Integral Function the Model has to Predict:

In [6]:
F_infix = 'ln(cos(x + exp(x)) * sin(x**2 + 2) * exp(x) / x)'

Converts **F_infix** to a type that can be used inside SymPy:

In [7]:
F = sympy.sympify(F_infix, locals=env.local_dict)
F

log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)

Get $f$, or $F'$, that the model will take as input:

In [8]:
f = F.diff(x)
f

x*(2*exp(x)*cos(x + exp(x))*cos(x**2 + 2) - (exp(x) + 1)*exp(x)*sin(x + exp(x))*sin(x**2 + 2)/x + exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x - exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x**2)*exp(-x)/(sin(x**2 + 2)*cos(x + exp(x)))

### Compute Prefix Representations

In [9]:
F_prefix = env.sympy_to_prefix(F)
f_prefix = env.sympy_to_prefix(f)
print(f"F with Prefix Notation: {F_prefix}")
print(f"f with Prefix Notation: {f_prefix}")

F with Prefix Notation: ['ln', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2']
f with Prefix Notation: ['mul', 'x', 'mul', 'pow', 'cos', 'add', 'x', 'exp', 'x', 'INT-', '1', 'mul', 'pow', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'INT-', '1', 'mul', 'add', 'mul', 'INT+', '2', 'mul', 'cos', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'exp', 'x', 'add', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'add', 'INT+', '1', 'exp', 'x', 'mul', 'exp', 'x', 'mul', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'sin', 'add', 'x', 'exp', 

### Encode Input

Clean prefix expressions before they are converted to PyTorch.

Examples:
- f x  -> Y
- derivative f x x  -> Y'

In [10]:
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
print(f"f Clean Prefix Notation: {x1_prefix}")

f Clean Prefix Notation: ['sub', "Y'", 'mul', 'x', 'mul', 'pow', 'cos', 'add', 'x', 'exp', 'x', 'INT-', '1', 'mul', 'pow', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'INT-', '1', 'mul', 'add', 'mul', 'INT+', '2', 'mul', 'cos', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'exp', 'x', 'add', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'add', 'INT+', '1', 'exp', 'x', 'mul', 'exp', 'x', 'mul', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'sin', 'add', 'x', 'exp', 'x', 'exp', 'mul', 'INT-', '1', 'x']


Create a PyTorch LongTensor for storing $f$ as a sequence of indexes based on prefix clean notation "words":

In [11]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
x1.transpose(0, 1)

tensor([[ 0, 67, 79, 54, 12, 54, 55, 40, 33, 12, 48, 12, 72, 82, 54, 55, 64, 33,
         71, 83, 55, 12, 71, 83, 72, 82, 54, 33, 54, 71, 83, 54, 40, 33, 71, 83,
         55, 12, 71, 83, 54, 40, 33, 12, 48, 12, 48, 12, 33, 54, 55, 12, 72, 82,
         54, 40, 33, 12, 48, 12, 54, 48, 12, 64, 33, 71, 83, 55, 12, 71, 83, 33,
         54, 72, 82, 54, 55, 12, 72, 83, 54, 40, 33, 12, 48, 12, 54, 48, 12, 64,
         33, 71, 83, 55, 12, 71, 83, 54, 72, 82, 54, 55, 12, 72, 82, 54, 33, 71,
         82, 48, 12, 54, 48, 12, 54, 64, 33, 71, 83, 55, 12, 71, 83, 64, 33, 12,
         48, 12, 48, 54, 72, 82, 12,  0]])

Move PyTorch tensors to CUDA (GPU):

In [12]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

Encodes the “meaning” of the input sequence into a single vector, with the Encoder of the Model:

In [13]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

tensor([[[ 0.0124, -0.0057,  0.0175,  ...,  0.0161, -0.0029, -0.0005],
         [ 0.0356, -0.0296,  0.0336,  ...,  0.0053,  0.0132, -0.0042],
         [ 0.0298, -0.0290,  0.0005,  ..., -0.0346, -0.0461,  0.0662],
         ...,
         [-0.0959,  0.0465,  0.0309,  ..., -0.0495, -0.0396,  0.0086],
         [ 0.0175,  0.0619,  0.0024,  ..., -0.0090, -0.0314,  0.0035],
         [ 0.0121,  0.0149,  0.0057,  ...,  0.0103, -0.0053,  0.0038]]],
       device='cuda:0')

### Decode with Beam Search

Instead of picking a single output (sequence, in this case an integral hypothesis), multiple highly probable choices are retained.

Declare beam size:

In [14]:
beam_size = 10

Takes the encoder output vector and outputs a sequence of "words" to recover the integral function $F$, using the
Decoder of the model.

In [15]:
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=200)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### Print the Results - Scores, Integrals Hypotheses

In [16]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Integral Hypothesis', 'Valid']
results = []

for score, sentence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sentence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
    hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

    # Check whether we recover "f" if we differentiate the hypothesis
    # Note that sometimes, SymPy fails to show that hyp' - f == 0, and the result is considered as invalid, although it may be correct
    validation = "YES" if sympy.simplify(hyp_sympy - F) == 0 else "NO"

    # Prepare results
    results.append([score, hyp_sympy, validation])

# Print results
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns)


,Score,Integral Hypothesis,Valid
1,-0.000029,log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x),YES
2,-0.284752,log(exp(x)*sin((x**3 + 2*x)/x)*cos(x + exp(x))/x),YES
3,-0.285919,log(exp(x)*sin(x*(x + 2/x))*cos(x + exp(x))/x),YES
4,-0.357943,log(exp(x)*sin(x*(x + 1) - x + 2)*cos(x + exp(x))/x),YES
5,-0.379522,log(exp(x)*sin(x**2*(x + 2/x))*cos(x + exp(x))/x),NO
6,-0.380337,log(exp(x)*sin(x**2 + 2)*cos(x + sinh(x) + cosh(x))/x),YES
7,-0.395180,atan(tan(log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x))),NO
8,-0.396895,log(exp(x)*sin(x*(x - 1) + x + 2)*cos(x + exp(x))/x),YES
9,-0.432026,log(exp(x)*sin((x**2 + 2)**2)*cos(x + exp(x))/x),NO
10,-0.445381,log(exp(x)*sin(x**2 + 2*x)*cos(x + exp(x))/x),NO
